<a href="https://colab.research.google.com/github/HiteshAI/face-recognition-demo/blob/master/face_recgnition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an example of running face recognition on a single image
and drawing a box around each person that was identified.

# **Import necessary libraries**

In [0]:
from PIL import Image, ImageDraw
from IPython.display import display
import numpy as np
!sudo pip install -v --install-option="--no" --install-option="DLIB_USE_CUDA" dlib
!pip3 install face_recognition
import face_recognition

In [0]:
detect_image = 'all3.jpg'
face_1 = 'kp.jpg'
face_2 = 'pd.jpg'
face_3 = 'deuba.jpg'


# **Encode known faces**

In [0]:
# Load a first picture and learn how to recognize it.
kp_image = face_recognition.load_image_file(face_1)
kp_face_encoding = face_recognition.face_encodings(kp_image)[0]

# Load second sample picture and learn how to recognize it.
pd_image = face_recognition.load_image_file(face_2)
pd_face_encoding = face_recognition.face_encodings(pd_image)[0]

# Load third sample picture and learn how to recognize it.
deuba_image = face_recognition.load_image_file(face_3)
deuba_face_encoding = face_recognition.face_encodings(deuba_image)[0]


# **Create array of encodings and labels for known faces**

In [0]:
# Create arrays of known face encodings and their names
known_face_encodings = [
    kp_face_encoding,
    pd_face_encoding,
    deuba_face_encoding
]
known_face_names = [
    'KP Oli',
    'Prachanda',
    'Deuba'
]
print('Learned encoding for', len(known_face_encodings), 'images.')

# **Load unknown image, find faces and encode them**

In [0]:
# Load an image with an unknown face
unknown_image = face_recognition.load_image_file(detect_image)

# Find all the faces and face encodings in the unknown image
face_locations = face_recognition.face_locations(unknown_image)
face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

# Convert the image to a PIL-format image so that we can draw on top of it with the Pillow library
# See http://pillow.readthedocs.io/ for more about PIL/Pillow
pil_image = Image.fromarray(unknown_image)
# Create a Pillow ImageDraw Draw instance to draw with
draw = ImageDraw.Draw(pil_image)


# **Comparision**

In [0]:
# Loop through each face found in the unknown image
for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
    # See if the face is a match for the known face(s)
    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
    name = "Unknown"
    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
    best_match_index = np.argmin(face_distances)

    if matches[best_match_index]:
        name = known_face_names[best_match_index]

    # Draw a box around the face using the Pillow module
    draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

    # Draw a label with a name below the face
    text_width, text_height = draw.textsize(name)
    draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
    draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))


# Remove the drawing library from memory as per the Pillow docs
del draw

# Display the resulting image
display(pil_image)

# **How it works**

If you want to learn how face location and recognition work instead of depending on a black box library, read this article https://medium.com/@ageitgey/machine-learning-is-fun-part-4-modern-face-recognition-with-deep-learning-c3cffc121d78